## Two-Way Model
### Data

In [62]:
using Distributions
n = 1000000
pA = 1000
pB = 500
A = sample([1:pA],n)
B = sample([1:pB],n)
α = randn(pA)
β = randn(pB)

### Computing sparse-stored X'X without matrix multiplication

In [7]:
p = 1 + pA + pB
XPX = spzeros(p,p)

1501x1501 sparse matrix with 0 Float64 entries:

In [12]:
posMu = 1
@time for i=1:size(A,1)           #B has the same number of rows!
    posA = 1 + A[i]
    posB = 1 + pA + B[i]
    XPX[posMu,posMu] += 1.0
    XPX[posMu,posA]  += 1.0
    XPX[posMu,posB]  += 1.0
    XPX[posA,posMu]  += 1.0
    XPX[posA,posA]   += 1.0
    XPX[posA,posB]   += 1.0
    XPX[posB,posMu]  += 1.0
    XPX[posB,posA]   += 1.0
    XPX[posB,posB]   += 1.0
end 

elapsed time: 2.875017448 seconds (399666120 bytes allocated, 23.46% gc time)


### Computing sparse-stored X'X as product of sparse-stored (X' and X)

In [79]:
ii  = 1:n
XA  = sparse(ii,A,1.0)
XB  = sparse(ii,B,1.0)
jj  = fill(1,n)
XMu = sparse(ii,jj,1.0)
@time X = [XMu XA XB];

elapsed time: 0.143970835 seconds (48025184 bytes allocated, 92.48% gc time)


In [44]:
@time XPX = X'X;

elapsed time: 0.636161008 seconds (122031248 bytes allocated, 12.35% gc time)


In [66]:
y = XA*α + XB*β + randn(n);

In [68]:
@time XPy = X'y;

elapsed time: 0.018238806 seconds (12160 bytes allocated)


In [70]:
@time sol = XPX\XPy;

CHOLMOD warning: matrix not positive definite
elapsed time: 0.881579143 seconds (137733512 bytes allocated, 29.08% gc time)


In [72]:
[XPX*sol XPy]

1501x2 Array{Float64,2}:
 87283.8    87283.8  
  1052.43    1052.43 
  -907.348   -907.348
 -1035.02   -1035.02 
   757.389    757.389
 -1461.79   -1461.79 
   460.433    460.433
  -832.001   -832.001
  1968.13    1968.13 
   572.016    572.016
  -712.859   -712.859
   758.905    758.905
  -557.517   -557.517
     ⋮               
 -1129.72   -1129.72 
  -992.297   -992.297
   664.42     664.42 
  2945.02    2945.02 
  -292.96    -292.96 
  -402.337   -402.337
 -1736.06   -1736.06 
 -2651.45   -2651.45 
 -1361.9    -1361.9  
  1414.25    1414.25 
  -831.49    -831.49 
  1922.69    1922.69 

In [71]:
sum((XPy - XPX*sol).^2)

1.209099095543353e-16